### Installation

In [1]:
%%capture
# Skip restarting message in Colab
import sys; modules = list(sys.modules.keys())
for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None

!pip install unsloth vllm
!pip install --upgrade pillow

### Unsloth

Use `PatchFastRL` before all functions to patch GRPO and other RL algorithms!

In [1]:
from unsloth import FastLanguageModel, PatchFastRL
PatchFastRL("GRPO", FastLanguageModel)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


Load up `Llama 3.1 8B Instruct`, and set parameters

In [3]:
from unsloth import is_bfloat16_supported
import torch
max_seq_length = 1024 # Can increase for longer reasoning traces
lora_rank = 32 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = 'Qwen/Qwen2.5-3B-Instruct', #"meta-llama/meta-Llama-3.1-8B-Instruct",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.6, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

INFO 02-25 20:35:22 __init__.py:207] Automatically detected platform cuda.
==((====))==  Unsloth 2025.2.15: Fast Qwen2 patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/qwen2.5-3b-instruct-unsloth-bnb-4bit with actual GPU utilization = 59.59%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 14.74 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 1024. Num Sequences = 192.
Unsloth: vLLM's KV Cache can use up to 6.48 GB. Also swap space = 2 GB.
WARNING 02-25 20:35:24 config.py:2448] Casting torch.bfloat16 to torch.float16.
INFO 02-25 20:35:36 config.py:549] This model suppo

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 02-25 20:35:50 model_runner.py:1115] Loading model weights took 2.2160 GB
INFO 02-25 20:35:50 punica_selector.py:18] Using PunicaWrapperGPU.
INFO 02-25 20:35:54 worker.py:267] Memory profiling takes 2.87 seconds
INFO 02-25 20:35:54 worker.py:267] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.60) = 8.78GiB
INFO 02-25 20:35:54 worker.py:267] model weights take 2.22GiB; non_torch_memory takes 0.05GiB; PyTorch activation peak memory takes 1.05GiB; the rest of the memory reserved for KV Cache is 5.47GiB.
INFO 02-25 20:35:54 executor_base.py:111] # cuda blocks: 9965, # CPU blocks: 3640
INFO 02-25 20:35:54 executor_base.py:116] Maximum concurrency for 1024 tokens per request: 155.70x
INFO 02-25 20:35:56 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error oc

Capturing CUDA graph shapes: 100%|██████████| 27/27 [00:54<00:00,  2.03s/it]

INFO 02-25 20:36:51 model_runner.py:1562] Graph capturing finished in 55 secs, took 0.62 GiB
INFO 02-25 20:36:51 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 60.58 seconds



Unsloth 2025.2.15 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


### Data Prep
<a name="Data"></a>

We directly leverage [@willccbb](https://gist.github.com/willccbb/4676755236bb08cab5f4e54a0475d6fb) for data prep and all reward functions. You are free to create your own!

In [4]:
from datasets import load_dataset

dataset_id = "Jiayi-Pan/Countdown-Tasks-3to4"
dataset = load_dataset(dataset_id, split="train")
dataset = dataset.shuffle(seed=42).select(range(50000))

SYSTEM_PROMPT = """
Respond using this structure:
<think>
Provide a concise, step-by-step explanation.
</think>
<answer>
Present your final equation.
</answer>
"""

def generate_r1_prompt(numbers, target):
    r1_prefix = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user",
         "content": (
             f"Using the numbers {numbers}, form an equation equaling {target} using basic arithmetic (+, -, *, /). "
             "Each number must be used once. Provide a brief explanation in <think> and the final equation in <answer>."
         )},
        {"role": "assistant", "content": "Let me solve this step by step.\n<think>"}
    ]
    return {"prompt": tokenizer.apply_chat_template(r1_prefix, tokenize=False, continue_final_message=True), "target": target}

# convert our dataset to the r1 prompt
dataset = dataset.map(lambda x: generate_r1_prompt(x["nums"], x["target"]))

# split the dataset into train and test
train_test_split = dataset.train_test_split(test_size=0.1)

train_dataset = train_test_split["train"]
test_dataset = train_test_split["test"]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [5]:
train_dataset[0]

{'target': 53,
 'nums': [67, 4, 77, 47],
 'prompt': '<|im_start|>system\n\nRespond using this structure:\n<think>\nProvide a concise, step-by-step explanation.\n</think>\n<answer>\nPresent your final equation.\n</answer>\n<|im_end|>\n<|im_start|>user\nUsing the numbers [67, 4, 77, 47], form an equation equaling 53 using basic arithmetic (+, -, *, /). Each number must be used once. Provide a brief explanation in <think> and the final equation in <answer>.<|im_end|>\n<|im_start|>assistant\nLet me solve this step by step.\n<think>'}

In [6]:
import re
import os

I = 1
def write_model_outputs_to_file(prompts, completions, target, output_dir="model_outputs3", **kwargs):
    global I
    try:
        os.makedirs(output_dir, exist_ok=True)
        filename = os.path.join(output_dir, f"output_{I}.txt")

        with open(filename, "w", encoding="utf-8") as file:
            for i, (p, c, a) in enumerate(zip(prompts, completions, target), 1):
                file.write(f"Example {i}:\n")
                file.write("Prompt:\n")
                file.write(str(p).strip() + "\n")
                file.write("-" * 30 + "\n")
                file.write("Model Completion:\n")
                file.write(str(c).strip() + "\n")
                file.write("-" * 30 + "\n")
                file.write("Expected Answer:\n")
                file.write(str(a).strip() + "\n")
                file.write("=" * 50 + "\n\n")
        I += 1

    except Exception as e:
        print(f"Error writing to file: {e}")
        return None

    return 0

import re

def format_reward_func(completions, target, **kwargs):
    rewards = []
    for completion, gt in zip(completions, target):
        reward = 0.0

        # Ensure the synthetic <think> is always there if needed
        completion = "<think>" + completion

        # Extract sections
        think_sections = re.findall(r"<think>[\s\S]*?<\/think>", completion)
        answer_sections = re.findall(r"<answer>[\s\S]*?<\/answer>", completion)

        # Evaluate <think> section
        if len(think_sections) == 0:
            reward -= 0.25  # missing <think> tag
        elif len(think_sections) == 1:
            reward += 0.5   # ideal case
        else:
            reward -= 0.25  # penalize extra <think> sections

        # Evaluate <answer> section
        if len(answer_sections) == 0:
            reward -= 0.25  # missing <answer> tag
        elif len(answer_sections) == 1:
            reward += 0.5   # ideal case
        else:
            reward -= 0.25  # penalize extra <answer> sections

        rewards.append(reward)
    return rewards


def equation_reward_func(completions, target, nums, **kwargs):
    rewards = []
    for completion, gt, numbers in zip(completions, target, nums):
        reward = 0.0
        # Prepend <think> to ensure consistency
        completion = "<think>" + completion

        # Extract the <answer> content
        match = re.search(r"<answer>(.*?)<\/answer>", completion, re.DOTALL)
        if match is None:
            rewards.append(-0.5)  # Penalize missing <answer>
            continue

        answer_content = match.group(1).strip()

        # Check for an equals sign and split if present
        if '=' in answer_content:
            parts = answer_content.split('=')
            if len(parts) > 2:
                reward -= 0.2  # Penalize multiple '=' signs
            expression = parts[0].strip()
            displayed_result = parts[1].strip() if len(parts) >= 2 else None
        else:
            expression = answer_content
            displayed_result = None

        # Allow '=' along with numbers and arithmetic operators
        allowed_pattern = r'^[\d+\-*/().=\s]+$'
        if not re.match(allowed_pattern, answer_content):
            reward -= 0.3

        # Extract numbers from the expression (left-hand side only)
        used_numbers = [int(n) for n in re.findall(r'\d+', expression)]

        # Penalize extra numbers (those not in the provided list)
        extra_numbers = [n for n in used_numbers if n not in numbers]
        if extra_numbers:
            reward -= 0.2

        # Reward based on the fraction of required numbers used correctly
        common_numbers = [n for n in used_numbers if n in numbers]
        if common_numbers:
            reward += 0.5 * (len(common_numbers) / len(numbers))
        else:
            reward -= 0.2

        # Safely evaluate the expression
        try:
            result = eval(expression, {"__builtins__": None}, {})
            if abs(float(result) - float(gt)) < 1e-5:
                reward += 0.5  # Correct result
                # Optional: Bonus for matching displayed result if present
                if displayed_result is not None:
                    try:
                        if abs(float(displayed_result) - float(gt)) < 1e-5:
                            reward += 0.1
                    except Exception:
                        pass
            else:
                reward -= 0.1  # Incorrect computed result
        except Exception:
            reward -= 0.3  # Evaluation error

        rewards.append(reward)
    return rewards

<a name="Train"></a>
### Train the model

Now set up GRPO Trainer and all configurations!

In [7]:
from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    use_vllm = True, # use vLLM for fast inference!
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "paged_adamw_8bit",
    logging_steps = 1,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 2, # Decrease if out of memory
    max_prompt_length = 256,
    max_completion_length = 1024,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 400,
    save_steps = 250,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 2


And let's run the trainer! If you scroll up, you'll see a table of rewards. The goal is to see the `reward` column increase!

You might have to wait 150 to 200 steps for any action. You'll probably get 0 reward for the first 100 steps. Please be patient!

| Step | Training Loss | reward    | reward_std | completion_length | kl       |
|------|---------------|-----------|------------|-------------------|----------|
| 1    | 0.000000      | 0.125000  | 0.000000   | 200.000000        | 0.000000 |
| 2    | 0.000000      | 0.072375  | 0.248112   | 200.000000        | 0.000000 |
| 3    | 0.000000      | -0.079000 | 0.163776   | 182.500000        | 0.000005 |


In [ ]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        format_reward_func,
        equation_reward_func,
        write_model_outputs_to_file,
    ],
    args = training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 45,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 1
\        /    Total batch size = 2 | Total steps = 400
 "-____-"     Number of trainable parameters = 59,867,136


Step,Training Loss,reward,reward_std,completion_length,kl,rewards / format_reward_func,rewards / equation_reward_func,rewards / write_model_outputs_to_file
1,-0.000000,0.308333,0.789603,548.000000,0.000000,0.625000,-0.316667,0.000000
2,0.000000,0.500000,0.000000,293.000000,0.000000,1.000000,-0.500000,0.000000
3,0.000000,0.500000,0.000000,487.000000,0.000013,1.000000,-0.500000,0.000000
4,0.000000,0.500000,0.000000,321.500000,0.000010,1.000000,-0.500000,0.000000
5,0.000000,0.500000,0.000000,252.000000,0.000013,1.000000,-0.500000,0.000000
6,0.000000,1.033333,0.754247,227.000000,0.000014,1.000000,0.033333,0.000000
7,0.000000,0.500000,0.000000,475.500000,0.000014,1.000000,-0.500000,0.000000
8,0.000000,0.500000,0.000000,436.000000,0.000013,1.000000,-0.500000,0.000000
9,0.000000,0.500000,0.000000,325.000000,0.000012,1.000000,-0.500000,0.000000
10,0.000000,0.500000,0.000000,212.000000,0.000011,1.000000,-0.500000,0.000000


<a name="Inference"></a>
### Inference
Now let's try the model we just trained! First, let's first try the model without any GRPO trained:

In [ ]:
text = tokenizer.apply_chat_template([
    {"role" : "user", "content" : "Calculate pi."},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output

Processed prompts: 100%|██████████| 1/1 [00:23<00:00, 23.78s/it, est. speed input: 1.64 toks/s, output: 19.94 toks/s]


'Calculating pi to a large number of decimal places is a complex task that requires a computational approach, rather than a simple mathematical formula. Here\'s a way to calculate pi using the Monte Carlo method, which is an approximation method that uses random numbers to estimate the value of pi:\n\n**The Monte Carlo Method**\n\nThe Monte Carlo method is based on the idea of simulating the probability of a random walk across a square and circle. Here\'s the basic idea:\n\n1. Draw a square and a circle on a piece of paper.\n2. Generate random points within the square.\n3. Count the proportion of points that fall within the circle.\n4. The ratio of points within the circle to the total number of points is approximately equal to the ratio of the area of the circle to the area of the square, which is pi.\n\n**Mathematical Formulation**\n\nLet\'s denote the following variables:\n\n*   `N`: the number of random points generated\n*   `n`: the number of points within the circle\n*   `pi_appr

And now with the LoRA we just trained with GRPO - we first save the LoRA first!

In [ ]:
model.save_lora("grpo_saved_lora")

Now we load the LoRA and test:

In [ ]:
text = tokenizer.apply_chat_template([
    {"role" : "system", "content" : SYSTEM_PROMPT},
    {"role" : "user", "content" : "Calculate pi."},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

output

Processed prompts: 100%|██████████| 1/1 [00:23<00:00, 23.29s/it, est. speed input: 2.62 toks/s, output: 19.41 toks/s]


"<reasoning>\nPi (π) is an irrational number that represents the ratio of a circle's circumference to its diameter. It is approximately equal to 3.14159, but its decimal representation goes on indefinitely without repeating.\n\nTo calculate pi, we can use various mathematical formulas and methods, such as the Leibniz formula, the Gregory-Leibniz series, or the Monte Carlo method. However, these methods are not practical for obtaining a high degree of accuracy.\n\nA more practical approach is to use the Bailey-Borwein-Plouffe (BBP) formula, which is a spigot algorithm that allows us to calculate any digit of pi without having to compute the preceding digits.\n\nAnother method is to use the Chudnovsky algorithm, which is a fast and efficient method for calculating pi to a high degree of accuracy.\n\nFor simplicity, we can use the first few terms of the BBP formula to estimate pi:\nπ = 3 + 1/(4/3 - 1/(4/3 - 1/(4/3 - ...))\n\nLet's use this simplified formula to estimate pi:\n\nπ ≈ 3 + 1/(

Our reasoning model is much better - it's not always correct, since we only trained it for an hour or so - it'll be better if we extend the sequence length and train for longer!

<a name="Save"></a>
### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Llama 3.2 Conversational notebook. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(1B_and_3B)-Conversational.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
